**Predicting Presence of Diabetes with Age, BMI, and Blood Sugar Level**

**Introduction:**


Diabetes is a chronic disease that affects the body's ability to turn food into energy. Normally after consumption, glucose is released into the bloodstream causing the pancreas to release insulin. This provides a signal so that our cells can utilize the newly acquired blood sugar for energy. Diabetics, however, are unable to produce insulin or cannot utilize it effectively, leading to major health problems. Unfortunately, many people suffer from this health problem.

This project aims to answer the question: can diabetes in an individual be predicted accurately using their age, BMI, and blood glucose level?

The diabetes prediction data set contains medical information about patients' diabetes status (positive or negative) and other pertinent health information. There are 9 columns (containing categorical and numerical data): age, gender, BMI, hypertension, heart disease, smoking history, HbA1c level, blood glucose level, and diabetes status. In total, there are 100,000 participants and thus 100,000 rows available. 

**Methods & Results**

In [1]:
#loading packages
library(tidyverse)
library(tidymodels)
library(dplyr)
#library(themis)

Warning message:
“package ‘ggplot2’ was built under R version 4.3.2”
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.5.0     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ rsample      1.2.0
✔ dials        1.2.0     ✔ tune         1.1.2
✔ infer        1.0.5     ✔ workflows    1.1.3
✔ modeldata    1.2.0     ✔ workflowsets 1.0.1
✔ parsnip      1.1.1     ✔ yardstick    1.2.0
✔ recipes      1.0.8     

── Conflicts ───────────────────────────────────────── tidymodels_co

In [2]:
#set seed, loading dataset, and splitting into train and test sets

set.seed(1)

url <- "https://raw.githubusercontent.com/danialtaj0/Dataset/main/diabetes_prediction_dataset.csv"
diabetes_data <- read_csv(url) |>
                 mutate(diabetes = as_factor(diabetes))

diabetes_split <- initial_split(diabetes_data, prop = 0.75, strata = diabetes)
diabetes_train <- training(diabetes_split)
diabetes_test <- testing(diabetes_split)
head(diabetes_train)

Rows: 100000 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (2): gender, smoking_history
dbl (7): age, hypertension, heart_disease, bmi, HbA1c_level, blood_glucose_l...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<fct>
Female,80,0,1,never,25.19,6.6,140,0
Female,20,0,0,never,27.32,6.6,85,0
Female,44,0,0,never,19.31,6.5,200,1
Female,79,0,0,No Info,23.86,5.7,85,0
Female,32,0,0,never,27.32,5.0,100,0
Female,53,0,0,never,27.32,6.1,85,0


class_distribution <- diabetes_train |>
  group_by(diabetes) |>
  summarise(Count = n())

class_distribution

predictor_means <- diabetes_train |>
  summarise(
    mean_bmi = mean(bmi, na.rm = TRUE),
    mean_age = mean(age, na.rm = TRUE),
    mean_blood_glucose_level = mean(blood_glucose_level, na.rm = TRUE)
  )

predictor_means

# Filtering rows where there is missing data in any of the specified columns
rows_with_missing_data <- diabetes_train |>
  filter(is.na(bmi) | is.na(age) | is.na(blood_glucose_level) | is.na(diabetes))


print(rows_with_missing_data)

# Count the number of rows with missing data
num_rows_with_missing_data <- nrow(rows_with_missing_data)

# Print the count
missing_data <- tibble(n_rows_missing_data = num_rows_with_missing_data)
missing_data

#making histograms and scatterplot
options(repr.plot.height = 5, repr.plot.width = 10)

bmi_dist <- ggplot(diabetes_train, aes(x = bmi)) + 
    geom_histogram() + 
    theme(text = element_text(size = 12)) + 
    ggtitle("Distribution of BMI") 

blood_glucose_dist <- ggplot(diabetes_train, aes(x = blood_glucose_level)) + 
    geom_histogram() + 
    theme(text = element_text(size = 12)) + 
    ggtitle("Distribution of Blood Glucose Level")

age_dist <- ggplot(diabetes_train, aes(x = age)) + 
    geom_histogram() + 
    theme(text = element_text(size = 12)) + 
    ggtitle("Distribution of Age")

age_bmi <- ggplot(diabetes_train, aes(x = age, y = bmi, color = diabetes, shape = diabetes)) + 
    geom_point() + 
    theme(text = element_text(size = 12)) + 
    ggtitle("Presence of Diabetes in Relation to Age and BMI")

bmi_dist
blood_glucose_dist
age_dist
age_bmi

In [ ]:
#preforming cross validation to find the optimal k value
diabetes_recipe <- recipe(diabetes ~ bmi + age + blood_glucose_level, data = diabetes_train) |>
    step_scale(all_predictors()) |> 
    step_center(all_predictors()) #|> 
    #step_upsample(over_ratio = 1, skip = FALSE) 

diabetes_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |> 
    set_engine("kknn") |> 
    set_mode("classification") 

diabetes_vfold <- vfold_cv(diabetes_train, v = 2, strata = diabetes)
k_vals = tibble(neighbors = 1:10) 

cv_results <- workflow() |> 
    add_model(diabetes_spec) |> 
    add_recipe(diabetes_recipe) |> 
    tune_grid(resamples = diabetes_vfold, grid = k_vals) |> 
    collect_metrics() |> 
    filter(.metric == "accuracy") |> 
    select(neighbors, mean) 
cv_results

**Discussion**

**References**